# GPS Clean Test

Testing the code for the GPS Clean app, which applies the pre-learned model to predict a new trace, provided in the GPX format, and corrects it using the Kalman Filter technique on the points recognised as outliers. 

In [1]:
import gpxpy
import gpxpy.gpx
import numpy as np
import gpsclean_transform as gt
import FullTraining as ft
import tensorflow as tf
import Correction as tc
from geojson import Feature, LineString, FeatureCollection, dump
import argparse

In [2]:
import importlib
importlib.reload(gt)
importlib.reload(ft)
importlib.reload(tc)

<module 'Correction' from 'C:\\Users\\Davide\\Documents\\unibz\\2_Master\\Tesi\\TransformAndModelling\\gpsclean\\Correction.py'>

Load a trace from a GPX file and create arrays of points and timestamps. 

In [3]:
labels_dict = {
    0 : 'Correct',
    1: 'Pause',
    2: 'Outlier',
    3: 'Indoor',  
}

In [4]:
gpx_file = open('Colle.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

In [5]:
points = []
times = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            points.append(np.array([point.longitude, point.latitude, point.elevation, point.time]))
            times.append(point.time)
            
points = np.array(points)
times = np.array(times)

In [6]:
points.shape

(2432, 4)

Now we need to create the deltas

In [7]:
deltas = gt.create_deltas(points, times)

In [8]:
deltas.shape

(2431, 4)

Loading the model

In [9]:
model = tf.keras.models.load_model("../models/model_42t_traces.h5")

Then we can predict the trace obtaining so the predictions of each point

In [10]:
segments, indices, predictions = ft.predict_trace(model, deltas, 15, 2)

(1216, 15, 4)
(1216, 15)
Predictions shape:  (1216, 15)


Compress predictions

In [11]:
pred_points = ft.compress_trace_predictions(predictions, indices, 4)

In [12]:
pred_points.shape

(2431,)

Correction part 1: removing pauses

In [13]:
full_pred_points = np.insert(pred_points, 0, 0)

In [14]:
reduced_points, reduced_times, reduced_delta, reduced_predictions, original_trace, original_times = tc.remove_pauses(points[:,:3], 
                                                                                                                     times, full_pred_points, None)

Coords:  2432 , Preds:  (2432,)


Correction part: now using Kalman filters

In [15]:
kalman_trace, kalman_times = tc.kalman_smoothing(reduced_points, reduced_times, reduced_predictions)

Points shape:  (2346, 3)
Filtered points:  (2346, 3)


In [129]:
reduced_points

array([[11.335952309891582, 46.49373251013458, 262.3999938964844],
       [11.33598298765719, 46.49378766305745, 262.6000061035156],
       [11.336032021790743, 46.49382412433624, 262.79998779296875],
       [11.336093042045832, 46.49382144212723, 262.79998779296875],
       [11.336106704548001, 46.49381406605244, 262.79998779296875],
       [11.336111901327968, 46.49376536719501, 262.6000061035156],
       [11.33612816222012, 46.49378632195294, 262.6000061035156]],
      dtype=object)

In [128]:
kalman_trace

array([[ 11.33595231,  46.49373251, 262.3999939 ],
       [ 11.33598299,  46.49378766, 262.6000061 ],
       [ 11.33603202,  46.49382412, 262.79998779],
       [ 11.33609304,  46.49382144, 262.7999878 ],
       [ 11.3361067 ,  46.49381407, 262.79998779],
       [ 11.3361119 ,  46.49376537, 262.60000611],
       [ 11.33612816,  46.49378632, 262.6000061 ]])

Now re-create cleaned gpx file

In [16]:
gpx = gpxpy.gpx.GPX()

# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

#loop over points and add
for i in range(kalman_trace.shape[0]):
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(longitude=kalman_trace[i][0], latitude=kalman_trace[i][1], 
                                                      elevation=kalman_trace[i][2], time=kalman_times[i]))

In [17]:
with open('Colle_cleaned.gpx', 'w') as f:
    f.write(gpx.to_xml())

And let's create a GeoJson file with the original trace and the derived annotations

In [18]:
geojson_coords = []
geojson_times = []

for i in range(points.shape[0]):
    geojson_coords.append((points[i][0], points[i][1], points[i][2]))
    geojson_times.append(times[i].isoformat())

In [19]:
annotations = []

cur_start = -1
cur_end = -1
cur_type = 0
cur_started = False

for i in range(full_pred_points.shape[0]):
    
    if full_pred_points[i] > 0:
        if cur_started:
            if full_pred_points[i] == cur_type:
                cur_end = i
            else:
                annotations.append({'start' : cur_start, 'end' : cur_end, 'annotation' : labels_dict[cur_type]})
                cur_start = i
                cur_end = i
                cur_type = full_pred_points[i]
                cur_started = True
        else:
            cur_start = i
            cur_end = i
            cur_type = full_pred_points[i]
            cur_started = True
    else:
        if cur_started:
            annotations.append({'start' : cur_start, 'end' : cur_end, 'annotation' : labels_dict[cur_type]})
        
        cur_started = False
        

if cur_started:
    annotations.append({'start' : cur_start, 'end' : cur_end - 1, 'annotation' : labels_dict[cur_type]})

In [122]:
annotations

[{'start': 5, 'end': 5, 'annotation': 'Pause'},
 {'start': 8, 'end': 50, 'annotation': 'Pause'}]

In [20]:
properties = {'coordTimes' : geojson_times, 'annotations' : annotations}

In [21]:
line = LineString(geojson_coords)
feature = Feature(geometry=line, properties = properties)

In [22]:
feature_collection = FeatureCollection([feature])

In [126]:
feature_collection

{"features": [{"geometry": {"coordinates": [[11.335952, 46.493733, 262.399994], [11.335983, 46.493788, 262.600006], [11.336032, 46.493824, 262.799988], [11.336093, 46.493821, 262.799988], [11.336107, 46.493814, 262.799988], [11.336113, 46.493765, 262.600006], [11.336112, 46.493765, 262.600006], [11.336128, 46.493786, 262.600006], [11.336138, 46.493808, 262.799988], [11.336154, 46.493822, 262.799988], [11.336185, 46.493841, 262.799988], [11.336188, 46.493841, 262.799988], [11.336195, 46.493838, 262.799988], [11.336209, 46.493838, 262.799988], [11.336214, 46.49383, 262.799988], [11.33621, 46.49383, 262.799988], [11.336182, 46.493845, 262.799988], [11.336219, 46.493786, 262.600006], [11.336253, 46.493791, 262.600006], [11.336256, 46.493793, 262.600006], [11.336256, 46.493794, 262.600006], [11.336281, 46.493802, 262.600006], [11.336269, 46.493807, 262.799988], [11.336258, 46.49381, 262.799988], [11.33625, 46.493811, 262.799988], [11.336242, 46.493812, 262.799988], [11.336239, 46.493811, 26

In [23]:
with open('Colle_predicted.geojson', 'w') as f:
    dump(feature_collection, f)